In [2]:
!date

Fri Mar 22 13:34:51 EDT 2024


In [3]:
pwd

'/mmfs1/data/aglinska/pub-CVAE-sim-neuroanatomy/Code'

In [ ]:
import os
import numpy as np
from tqdm import tqdm

In [ ]:
def get_random_string():
    import string
    import random
    return ''.join([random.choice(string.ascii_letters) for _ in range(10)])

def safe_mkdir(dirpath):
    if not os.path.exists(dirpath):
        os.mkdir(dirpath)

In [ ]:
analysis_name = 'CVAE_3_subtype_N1000_'+get_random_string()
print(analysis_name)

In [ ]:
safe_mkdir(f'../scratch/{analysis_name}')

In [ ]:
save_dir_stage1 = f'../scratch/{analysis_name}/stage_1'
save_dir_stage2 = f'../scratch/{analysis_name}/stage_2'

df_fn = '../Data/synth-data-03-N1000.csv'
data_fn = '../Data/synth-data-03-N1000.npy'

In [ ]:
%%time
for iter in tqdm(range(50)):
    os.system(f'python initialize_CVAE.py {save_dir_stage1} {data_fn} {df_fn} {iter} 10 None')

In [ ]:
files_varexp = [file for file in os.listdir(save_dir_stage1) if file.startswith('varexp_attmp_')];files_varexp.sort()

varexps = np.array([np.load(os.path.join(save_dir_stage1,f))[-1] for f in files_varexp])

file_nums = np.array([int(f.replace('varexp_attmp_','').replace('.npy','')) for f in files_varexp])
ORD = np.argsort(varexps)[-1::-1]
[print(f'{files_varexp[o]} {varexps[o]:.2f}') for o in ORD[0:20]];
IDs_best = file_nums[ORD[0:20]]
stage_2_weights = [os.path.join(save_dir_stage1,f'CVAE_init_attmp_{val}') for val in IDs_best]

In [ ]:
%%time
for iter in tqdm(range(20)):
    os.system(f'python initialize_CVAE.py {save_dir_stage2} {data_fn} {df_fn} {iter} 100 {stage_2_weights[iter]}')